In [8]:
# !pdftotext -f 244 -l 244 "../EXAMPLE/Yes Bank -A (32448549).pdf" "../EXAMPLE/output.txt" 
# !pdftotext -f 57 -l 57 "../Samples/Non-Image/143529570.pdf" "outputs/output.txt" 
# !pdftotext -f 57 -l 57 "../Samples/Non-Image/143529570.pdf" "outputs/output.html" -htmlmeta -layout
# !pdftotext -f 43 -l 43 "../Samples/Non-Image/140258305.pdf" "outputs/output.html" -htmlmeta -layout

# !pdftotext -f 147 -l 147 "../Samples/Non-Image/178540207.pdf" "outputs/output.html" -htmlmeta -layout #Problem in text

In [20]:
import re, os, sys, glob
from pathlib import Path
from collections import Counter
import pickle
import PyPDF2

import pandas as pd
import numpy as np

from sklearn import metrics

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = None
np.warnings.filterwarnings('ignore')

In [21]:
# os.system(cmd)
model = pickle.load(open('models/nb.pkl', 'rb'))

In [22]:
import os, glob
import shutil

def delete_contents(path, delete_files=True, delete_folders=True, delete_itself=False):
    if os.path.isdir(path):
        if delete_itself:
            shutil.rmtree(path)
        else:
            if delete_files:
                files = glob.glob('%s/*.*'%path)
                for f in files:
                    os.remove(f)
            if delete_folders:        
                folders = glob.glob('%s/*'%path)
                for f in folders:
                    os.rmdir(f)

def create_folder(directory, delete_old=False):
    if delete_old:
        delete_contents(directory, delete_itself=True)
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            print('Directory created. ' + directory)
    except OSError:
        print('Directory exists. ' + directory)
        
def preprocessor(x):
    return re.sub(r'[0-9\,\n()]+', '', x).lower()

In [23]:
delete_contents('outputs/asd/')

In [24]:
pdf_folder = '../Samples/Non-Image'
filepath = '%s/140258305.pdf'%pdf_folder
filename = os.path.basename(filepath).split('.')[0]

folder = 'outputs/%s'%filename
create_folder(folder, delete_old=True)
folder_images = 'outputs/%s/images'%filename
create_folder(folder_images)
folder_html = 'outputs/%s/html'%filename
create_folder(folder_html)

pdfFileObj = open(filepath, 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
pages=pdfReader.numPages
print('No. of pages: %s'%pages)
# file = open('outputs/%s.txt'%filename, "w", encoding='UTF-8')

df = pd.DataFrame()
page_text = []
table_type = []
probabilities = []

for i in range(1,pages):
    pageObj = pdfReader.getPage(i)
    text=pageObj.extractText()
    text = preprocessor(text)
    p = np.max(model.predict_proba([text]))
    if p>0.95:
        tb_type = model.predict([text])[0]
        print('Saving %s of page %s'%(tb_type, i))
        page_text.append(text[:20])
        table_type.append(tb_type)
        probabilities.append(p)
    #     file.write(convert)
        os.system("pdftotext -f %s -l %s %s %s -htmlmeta -layout" % (i, i, '%s/%s.pdf'%(pdf_folder, filename), '%s/%s-%s.html'%(folder_html, i, tb_type)))
# file.close()
pdfFileObj.close()

df['page_text'] = page_text
df['prob'] = probabilities
df['table_type'] = table_type
df[df.table_type.isin(['is', 'bs', 'cf'])]

Directory created. outputs/140258305
Directory created. outputs/140258305/images
Directory created. outputs/140258305/html
No. of pages: 109
Saving plaintext of page 4
Saving plaintext of page 5
Saving plaintext of page 9
Saving plaintext of page 14
Saving plaintext of page 25
Saving plaintext of page 27
Saving plaintext of page 32
Saving plaintext of page 35
Saving plaintext of page 36
Saving plaintext of page 39
Saving bs of page 40
Saving is of page 41
Saving cf of page 42
Saving cf of page 43
Saving note of page 46
Saving note of page 49
Saving note of page 52
Saving note of page 59
Saving note of page 62
Saving note of page 64
Saving bs of page 68
Saving is of page 69
Saving cf of page 70
Saving cf of page 71
Saving note of page 80
Saving note of page 86
Saving note of page 90
Saving plaintext of page 93
Saving plaintext of page 94
Saving plaintext of page 95
Saving plaintext of page 96
Saving plaintext of page 97
Saving plaintext of page 99
Saving plaintext of page 107


,page_text,prob,table_type
10,consolidated balance,0.999814,bs
11,consolidated stateme,0.997488,is
12,consolidated cash fl,0.999964,cf
13,particularsfor the y,0.972706,cf
20,balance sheet as at,0.999814,bs
21,statement of profit,0.996073,is
22,cash flow statement,0.999969,cf
23,particularsfor the y,0.976226,cf


In [25]:
df

,page_text,prob,table_type
0,management discussio,0.973231,plaintext
1,aptech indian techn,0.959494,plaintext
2,an economic point of,0.973449,plaintext
3,research & developme,0.952185,plaintext
4,annexure v to direct,0.990593,plaintext
5,other provisions:the,0.991991,plaintext
6,iii approval of emp,0.999167,plaintext
7,auditors™ certificat,0.995766,plaintext
8,certification by chi,0.974022,plaintext
9,control based on the,0.975387,plaintext
